In [3]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



df= pd.read_csv('/Users/ssanjua/Desktop/HENRY 2023/DataScience/DATAFT10/LABS/PI MLs/PI_ML_OPS_pau/Datasets/movies_dataset.csv')

/var/folders/71/4rl6w509079c20dtbvnqwf1c0000gn/T/ipykernel_12788/3778482349.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('/Users/ssanjua/Desktop/HENRY 2023/DataScience/DATAFT10/LABS/PI MLs/PI_ML_OPS_pau/Datasets/movies_dataset.csv')


In [4]:


# Seleccionar las columnas relevantes
df = df[['title', 'vote_average', 'popularity', 'overview']]

# Rellenar los valores faltantes en la columna 'overview' con una cadena vacía
df['overview'] = df['overview'].fillna('')

# Reducir el tamaño del conjunto de datos seleccionando una muestra aleatoria
df_sample = df.sample(n=10000, random_state=42)

# Crear una matriz de características utilizando TF-IDF en la columna 'overview'
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_sample['overview'])

# Reducir la dimensionalidad de la matriz utilizando descomposición de matrices
svd = TruncatedSVD(n_components=100, random_state=42)
latent_matrix = svd.fit_transform(tfidf_matrix)

# Calcular la similitud del coseno entre las películas de la muestra
cosine_sim = cosine_similarity(latent_matrix)

# Obtener los títulos y los índices de las películas de la muestra
titles = df_sample['title']
indices = pd.Series(df_sample.index, index=df_sample['title']).drop_duplicates()

# Función para obtener recomendaciones basadas en el título de una película
def get_recommendations2(title, cosine_sim=cosine_sim, top_n=5):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:top_n]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]



get_recommendations2('Toy Story')

43526              Small Crimes
4057     The Unbelievable Truth
16889         The Criminal Code
9987      The Amityville Horror
7908        Last Cannibal World
Name: title, dtype: object

In [3]:


# Crear una instancia de TfidfVectorizer con stop words en inglés
tfidf = TfidfVectorizer(stop_words='english')

# Rellenar los valores faltantes en la columna 'overview' con una cadena vacía
df['overview'] = df['overview'].fillna('')
dff=df.sample(n=10000, random_state=42)
# Aplicar la transformación TF-IDF a la columna 'overview'
tfidf_matrix = tfidf.fit_transform(dff['overview'])

# Calcular la similitud del coseno entre los documentos
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Crear un diccionario de índices de películas basado en los títulos
indices = pd.Series(dff.index, index=dff['title']).drop_duplicates()


def get_recommendations1(title, cosine_sim=cosine_sim, top_n=5):
    # Obtener el índice de la película según el título ingresado
    idx = indices[title]

    # Obtener los puntajes de similitud de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar los puntajes de similitud en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Filtrar las películas similares para evitar recomendaciones duplicadas o poco relevantes
    sim_scores = [x for x in sim_scores if x[1] > 0.0][:top_n]

    # Obtener los índices de las películas recomendadas
    movie_indices = [i[0] for i in sim_scores]

    # Devolver los títulos de las películas recomendadas
    return dff['title'].iloc[movie_indices]


get_recommendations1('Toy Story')

43526      Small Crimes
34828    The Here After
42513      Dear God No!
22510      Rhino Season
32735          Drishyam
Name: title, dtype: object

In [5]:


# Descargar los recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Crear una instancia de TfidfVectorizer con stop words en inglés
tfidf = TfidfVectorizer(stop_words='english', lowercase=True)

# Rellenar los valores faltantes en la columna 'overview' con una cadena vacía
df['overview'] = df['overview'].fillna('')

# Preprocesamiento de texto: limpiar y lematizar el texto
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^A-Za-z]', ' ', text)
    
    # Convertir a minúsculas
    text = text.lower()
    
    # Tokenizar el texto en palabras individuales
    words = text.split()
    
    # Eliminar las palabras de parada y lematizar el texto
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    # Unir las palabras nuevamente en una cadena
    text = ' '.join(words)
    
    return text

# Aplicar el preprocesamiento de texto a la columna 'overview'
df['overview'] = df['overview'].apply(preprocess_text)

# Seleccionar las columnas 'title', 'overview' y 'vote_average'
df_sample = df[['title', 'overview', 'vote_average']].sample(n=10000, random_state=42)

# Aplicar la transformación TF-IDF a la columna 'overview' en el subconjunto de datos
tfidf_matrix = tfidf.fit_transform(df_sample['overview'])

# Calcular la similitud del coseno utilizando el kernel lineal
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Crear un diccionario de índices de películas basado en los títulos
indices = pd.Series(df_sample.index, index=df_sample['title']).drop_duplicates()


def get_recommendations(title, cosine_sim=cosine_sim, top_n=5):
    # Obtener el índice de la película según el título
    idx = indices[title]
    
    # Obtener los puntajes de similitud de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar los puntajes de similitud en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Filtrar las películas similares para evitar recomendaciones duplicadas o poco relevantes
    sim_scores = [x for x in sim_scores if x[1] > 0.0][:top_n]
    
    # Obtener los índices de las películas recomendadas
    movie_indices = [i[0] for i in sim_scores]
    
    # Devolver los títulos de las películas recomendadas
    return df_sample['title'].iloc[movie_indices]

get_recommendations('Toy Story')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/ssanjua/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [6]:
import nltk
nltk.download()


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


: 

: 